## Problem 1: Generating Random Boolean Functions

The [Deutsch–Jozsa algorithm](https://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa) works with Boolean functions that accept a fixed number of Boolean inputs and return a single Boolean output.

### Problem Statement

Write a Python function `random_constant_balanced` that returns a randomly chosen function from the set of **constant** or **balanced** functions taking **four Boolean arguments** as inputs.

### Function Types

- **Constant functions**: Always return `True` OR always return `False` (2 total)
- **Balanced functions**: Return `True` for exactly half (8) of the 16 possible input combinations

### Approach

With 4 Boolean inputs, there are 2^4 = 16 possible input combinations. I'll use a truth table approach where:
1. Generate all 16 possible input combinations using [`itertools.product`](https://docs.python.org/3/library/itertools.html#itertools.product)
2. Randomly decide whether to create a constant or balanced function
3. Build a truth table mapping inputs to outputs
4. Return a function that uses this truth table

In [1]:
import itertools
import random

def random_constant_balanced():
    """
    Returns a randomly chosen function from the set of constant or balanced functions.
    Takes 4 Boolean arguments as inputs.
    
    Returns:
        A function that takes 4 boolean arguments and returns a boolean.
    """
    # Generate all 16 possible input combinations for 4 boolean variables
    all_inputs = list(itertools.product([False, True], repeat=4))
    
    # Randomly choose between constant or balanced function
    is_constant = random.choice([True, False])
    
    if is_constant:
        # Constant function: always return True or False
        constant_value = random.choice([True, False])
        truth_table = {inputs: constant_value for inputs in all_inputs}
    else:
        # Balanced function: return True for exactly 8 out of 16 combinations
        # Randomly select 8 combinations to return True
        true_outputs = random.sample(all_inputs, 8)
        truth_table = {inputs: (inputs in true_outputs) for inputs in all_inputs}
    
    # Return a function that uses the truth table
    def boolean_function(a, b, c, d):
        return truth_table[(a, b, c, d)]
    
    return boolean_function

In [2]:
func = random_constant_balanced()
print(func(True, False, True, False))  # Test with sample inputs

False
